# configurando drive

In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

# Deep Learning - parte 2: Convolutional Neural Network

Redes convolucionais são usadas principalmente para classificação de imagens.

## Como elas funcionam?

Filtros convolucionais são usados para extrair features de imagens. Vamos olhar um exemplo de como extrair bordas de imagens:

<img src="https://cdn-images-1.medium.com/max/1000/1*TAo3aselJNVwrLLr654Myg.gif" alt="drawing" width="800"/>

Nas redes convolucionais esses filtros são aplicados em várias camadas:

<img src="https://cdn-images-1.medium.com/max/1600/1*_34EtrgYk6cQxlJ2br51HQ.gif" width="700"/>

Os valores dos filtros são aprendidos, portanto a própria rede aprende quais características são relevantes.

![](https://adeshpande3.github.io/assets/deconvnet.png)


## Construindo uma CNN pra predizer lateralidade do raio-X

In [0]:
# Primeiro vamos definir os imports

import warnings
warnings.filterwarnings('ignore')


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time
import os
import datetime


from keras import datasets, Model
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras import Sequential
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras import backend
import tensorflow as tf

from keras.preprocessing.image import load_img, img_to_array


Using TensorFlow backend.


## Input de dados

Existem várias formas de inputar os dados para treinamento, vamos usar o `Image Data Generator` para ler as imagens a partir do disco

In [0]:
!ls /content/drive/images-chest-orientation

In [0]:
!ls /content/drive/images-chest-orientation/train

In [0]:
# aqui definimos as transformações que serão aplicadas na imagem e a % de dados 
# que serão usados para validação

data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.30)

In [0]:
# para criar os generators precisamos definir o path da pasta raiz com as imagens e o tamanho da BATCH SIZE

path = '/content/drive/images-chest-orientation/train/'
BATCH_SIZE = 50


train_generator = data_generator.flow_from_directory(path, shuffle=True, seed=13,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")

validation_generator = data_generator.flow_from_directory(path, shuffle=True, seed=13,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")

Vamos dar uma olhada em como ficaram as imagens:

In [0]:
plt.imshow(train_generator[3][0][0])
shape = train_generator[3][0][0].shape

In [0]:
plt.imshow(validation_generator[1][0][0])

## Definindo a arquitetura:

Vamos criar uma rede pequena que consiga identificar corretamente essas imagens:

In [0]:
def build_model(shape):
    '''
    Constroi as camadas da rede
    :return: modelo construido
    '''
    
    model = Sequential()

    # primeira camada adiciona o shape do input
    # adiciona a funcao de ativacao
    # padding define o output da camada, "same" eh mesmo tamanho
    # tamanho do kernel (mascara)
    # quantidade de filtros (neurônios)
    # também é possível alterar a inicializacao, bias, entre outros -- https://keras.io/layers/convolutional/#conv2d
    model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=shape, kernel_initializer='glorot_uniform'))
    #Tamanho do downsampling
    model.add(MaxPooling2D(pool_size=2))
    # Fracao das unidades que serao zeradas
    model.add(Dropout(0.3))

    # Segunda camada
    model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.3))

    # Da um reshape no output transformando em array
    model.add(Flatten())

    # Camada full-connected 
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    #Camada de saida com o resultado das classes
    model.add(Dense(2, activation='sigmoid'))

    return model

Outros tipos de camadas utilizadas:

MaxPooling:
Realiza o downsampling pós convolução.

<img src="https://developers.google.com/machine-learning/practica/image-classification/images/maxpool_animation.gif" width=350/>




Dropout:
Em cada época desativa aleatoriamente um % de neurônios. Evita overfitting
    
<img src="https://i0.wp.com/deeplearningbook.com.br/wp-content/uploads/2018/07/rede2.png?w=310" width=300/>

In [0]:
model = build_model(shape)
model.summary()

In [0]:
# Compila o modelo definindo: otimizador, metrica e loss function
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

## Treinamento

Como lemos os dados usando um generator, o fit do keras também será usando um `fit_generator`.

Também usaremos alguns `callbacks`: 
 - ModelCheckPoint para salvar o modelo que tiver o melhor loss durante o treinamento e,
 - EarlyStop para interromper o treinamento caso a rede pare de aprender (convergiu).

In [0]:
checkpoint = ModelCheckpoint('chest_orientation_model.hdf5', 
                             monitor='val_loss', 
                             verbose=1, mode='min', 
                             save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=0.001,
                                   patience=5, # geralmente colocamos no mínimo 5 epocas
                                   mode='min',
                                   verbose=1)

In [0]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch = train_generator.samples//BATCH_SIZE,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples//BATCH_SIZE,
                    epochs= 50,
                    callbacks=[checkpoint, early_stop]
                    )

## Avaliação:

Sempre importante separar uma quantidade de dados para testar o modelo no final

In [0]:
import glob

In [0]:
test_set = glob.glob('/content/drive/images-chest-orientation/test/**/*.jpg', recursive=True)

In [0]:
test_set

In [0]:
# temos que fazer o load do model que teve o melhor loss
model = load_model('chest_orientation_model.hdf5')

In [0]:
image_test = np.array([img_to_array(load_img(image_name, target_size=(256, 256), color_mode='rgb'))/255 for image_name in test_set])

In [0]:
y_pred = model.predict(image_test)

In [0]:
y_true = [0,0,0,0,0,1,1,1,1,1]
labels = ['Frente', 'Lateral']
figure = plt.figure(figsize=(20, 8))
for i in range(10):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    im = plt.imread(test_set[i])
    ax.imshow(im)
    predict_index = np.argmax(y_pred[i])
    true_index = y_true[i]
    # Set the title for each image
    ax.set_title("{} ({})".format(labels[predict_index], 
                                  labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))

## Transfer Learning

Nem sempre precisamos definir a arquitetura da nossa rede do zero. Para reconhecimento de imagens existem arquiteturas já definidas e pré-treinadas com o Imagenet - um dataset com 1.2 milhões de imagens e 1000 categorias.

![](https://cdn-images-1.medium.com/max/1400/1*n16lj3lSkz2miMc_5cvkrA.jpeg)
[referência](https://towardsdatascience.com/neural-network-architectures-156e5bad51ba)

Para usar essas redes utilizamos **transfer learning**, transferir os pesos já aprendidos nessas redes para uma tarefa.

In [0]:
from keras.applications import VGG16

In [0]:
def build_model(shape):
    '''
    Constroi as camadas da rede
    :return: modelo construido
    '''
    
    vgg = VGG16(weights = "imagenet", include_top=False, input_shape = shape)
    # congelando camadas que não iremos treinar.
    for layer in vgg.layers[5:]:
        layer.trainable = False
    
    x = vgg.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(2, activation='sigmoid')(x)
    
    model = Model(input = vgg.input, output = predictions)

    return model

In [0]:
model = build_model(shape)

In [0]:
model.summary()

In [0]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

checkpoint = ModelCheckpoint('chest_orientation_model.hdf5', 
                             monitor='val_loss', 
                             verbose=1, mode='min', 
                             save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss',
                                   min_delta=0.001,
                                   patience=5, # geralmente colocamos no mínimo 5 epocas
                                   mode='min',
                                   verbose=1)

model.fit_generator(generator=train_generator,
                    steps_per_epoch = train_generator.samples//BATCH_SIZE,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples//BATCH_SIZE,
                    epochs= 50,
                    callbacks=[checkpoint, early_stop]
                    )

- Vamos visualizar algumas classificações

In [0]:
y_pred = model.predict(image_test)
y_true = [0,0,0,0,0,1,1,1,1,1]
labels = ['Frente', 'Lateral']
figure = plt.figure(figsize=(20, 8))
for i in range(10):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    im = plt.imread(test_set[i])
    ax.imshow(im)
    predict_index = np.argmax(y_pred[i])
    true_index = y_true[i]
    # Set the title for each image
    ax.set_title("{} ({})".format(labels[predict_index], 
                                  labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))

## Razões de porque sua rede pode não estar funcionando:

https://blog.slavv.com/37-reasons-why-your-neural-network-is-not-working-4020854bd607